<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training4/blob/main/Words2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip wiki_zh_2019
#unzip the zipfile

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
import json
ALL_DATAS = []
filesList = {
    0:"AA",
    1:"AB",
    2:"AC",
    4:"AD",
    5:"AE",
    6:"AF",
    7:"AG",
    8:"AH",    
    9:"AI",
    10:"AJ",
    11:"AK",
    12:"AL",
    13:"AM"
}

for i in filesList:
  folderPath = "./wiki_zh/%s/" % filesList[i]
  files = listdir(folderPath)
  for i in files:
    curFilePath = folderPath + "%s"%i 
    print(curFilePath)
    with open(curFilePath) as f:
      for j in f.readlines():
        str2Json = json.loads(j)
        text = "".join(str2Json["text"].strip().split())
        ALL_DATAS.append(text)
      f.close()


In [ ]:
!pip install opencc-python-reimplemented

In [47]:
#convert text 
from opencc import OpenCC
#convert to zh-hk
ccConvertor = OpenCC("s2hk")
transChineseList = []
for i in ALL_DATAS:
  transChineseList.append(ccConvertor.convert(i))


In [55]:
#store convertd data to csv
import pandas as pd
datas = pd.DataFrame(transChineseList)
datas.to_csv("alldatas.csv",encoding="utf-8")

# cut words

In [56]:
!pip install jieba

In [58]:
#load stopwords list
import codecs 
fp = codecs.open("./stopwords.dat","r",encoding="utf-8")
contents = fp.read()
fp.close()

In [ ]:
import jieba
import jieba.analyse
cutWordsListDatas = []
x = 0;
for i in transChineseList:
  cutWordsListDatas.append(" ".join([word for word in jieba.cut(i,cut_all=False) if word not in contents]))
  if(x%5000 == 0):
    print(x)
  x+=1

In [98]:
cutWordsTxt = "cutWordsList2.txt"
with open(cutWordsTxt,"w") as fw:
  for data in cutWordsListDatas:
    fw.write(data)
    fw.write("\n")


# Word2vec for our text

In [ ]:
!pip install word2vec

In [ ]:
import word2vec
#change our words to vector
word2vec.word2vec("cutWordsList2.txt","cutWords2Vec.bin",size=300,verbose=True)

Running command: word2vec -train cutWordsList2.txt -output cutWords2Vec.bin -size 300 -window 5 -sample 1e-3 -hs 0 -negative 5 -threads 12 -iter 5 -min-count 5 -alpha 0.025 -debug 2 -binary 0 -cbow 1
Starting training using file cutWordsList2.txt
Vocab size: 890060
Words in train file: 124647462
Alpha: 0.004841  Progress: 80.64%  Words/thread/sec: 58.73k  